In [2]:
import re, nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [5]:
from nltk.tokenize import word_tokenize, sent_tokenize

words = word_tokenize(text.lower())
unique_words = set(words)

vocab = {word: i for i, word in enumerate(unique_words)}
print(vocab)

print()

seq_1 = sent_tokenize(text.lower())[0]
indexes = [vocab[word] for word in word_tokenize(seq_1)]
print(indexes)

{'note': 0, 'command': 1, 'only': 2, 'is': 3, 'preferences': 4, 'for': 5, 'stable': 6, 'install': 7, 'version': 8, 'run': 9, 'most': 10, 'that': 11, 'available': 12, 'libtorch': 13, 'of': 14, 'pytorch': 15, 'currently': 16, 'and': 17, 'tested': 18, 'c++': 19, 'select': 20, 'the': 21, '.': 22, 'represents': 23, 'your': 24, 'supported': 25}

[20, 24, 4, 17, 9, 21, 7, 1, 22]


1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [85]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [86]:
word_tokenize(sent_tokenize(text)[0].lower())

['select',
 'your',
 'preferences',
 'and',
 'run',
 'the',
 'install',
 'command',
 '.']

In [87]:
words = word_tokenize(text.lower())
print(words)

print()

unique_words = set(words)
print(unique_words)
print(len(unique_words))

vocab = {word: i for i, word in enumerate(unique_words)}

one_hot = torch.eye(len(vocab))

sentence_vectors = []
for token in word_tokenize(sent_tokenize(text)[0].lower()):
    index = vocab[token]
    vector = one_hot[index]
    sentence_vectors.append(vector)

sentence_vectors = torch.stack(sentence_vectors)

print(sentence_vectors.shape)
print(sentence_vectors)

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command', '.', 'stable', 'represents', 'the', 'most', 'currently', 'tested', 'and', 'supported', 'version', 'of', 'pytorch', '.', 'note', 'that', 'libtorch', 'is', 'only', 'available', 'for', 'c++']

{'that', 'is', 'c++', 'stable', 'note', 'available', 'only', 'tested', 'represents', 'preferences', 'of', 'libtorch', 'command', 'version', 'the', 'your', 'run', 'install', 'currently', 'select', 'and', 'for', 'most', 'pytorch', 'supported', '.'}
26
torch.Size([9, 26])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 

In [88]:
[word_tokenize(word)[0] for word in sent_tokenize(text)[0].split()]

['Select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [89]:
words = word_tokenize(text.lower())

unique_words = set(words)
print(len(unique_words))
print(unique_words)

vocab = {word: idx for idx, word in enumerate(unique_words)}
print(vocab)

sentence_idxs = [vocab[word] for word in word_tokenize(seq_1)]
print(len(sentence_idxs))
print(sentence_idxs)

embedding = nn.Embedding(26, len(unique_words))
sentence_vectors = embedding(torch.tensor(sentence_idxs))

print(sentence_vectors.shape)
print(sentence_vectors)

26
{'that', 'is', 'c++', 'stable', 'note', 'available', 'only', 'tested', 'represents', 'preferences', 'of', 'libtorch', 'command', 'version', 'the', 'your', 'run', 'install', 'currently', 'select', 'and', 'for', 'most', 'pytorch', 'supported', '.'}
{'that': 0, 'is': 1, 'c++': 2, 'stable': 3, 'note': 4, 'available': 5, 'only': 6, 'tested': 7, 'represents': 8, 'preferences': 9, 'of': 10, 'libtorch': 11, 'command': 12, 'version': 13, 'the': 14, 'your': 15, 'run': 16, 'install': 17, 'currently': 18, 'select': 19, 'and': 20, 'for': 21, 'most': 22, 'pytorch': 23, 'supported': 24, '.': 25}
9
[19, 15, 9, 20, 16, 14, 17, 12, 25]
torch.Size([9, 26])
tensor([[-4.6437e-02, -1.0412e+00, -6.2842e-01, -2.0072e+00,  7.2855e-01,
          1.4604e+00,  9.3112e-01, -1.4635e-01,  1.4755e+00, -3.6397e-01,
          1.3493e+00, -3.4245e-01,  2.2404e-01,  1.1051e+00, -6.6909e-01,
          1.6545e+00,  6.3852e-01,  4.9884e-01,  9.7245e-01,  1.8334e-02,
         -1.4559e+00,  1.5422e+00,  1.7896e-02,  9.2047

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [183]:
surnames = pd.read_csv("data/surnames.csv")
surnames['nationality'], _ = pd.factorize(surnames['nationality'])
surnames

,surname,nationality
0,Woodford,0
1,Coté,1
2,Kore,0
3,Koury,2
4,Lebzak,3
...,...,...
10975,Quraishi,2
10976,Innalls,0
10977,Król,12
10978,Purvis,0


In [184]:
X = surnames['surname'].str.lower()
y = surnames['nationality']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [185]:
class Vocab:
  def __init__(self, data):
    tokens = set()
    max_seq_len = 0
    for item in data:
        max_seq_len = max(max_seq_len, len(item))
        tokens.update(item)

    self.idx_to_token = {0: '<PAD>'}
    self.token_to_idx = {'<PAD>': 0}
    for idx, token in enumerate(tokens, start=1):
        self.idx_to_token[idx] = token
        self.token_to_idx[token] = idx
    self.vocab_len = len(self.idx_to_token)
    self.max_seq_len = max_seq_len

vocab = Vocab(X_train)
print(vocab.vocab_len)
print(vocab.idx_to_token)
print(vocab.max_seq_len)

54
{0: '<PAD>', 1: 'o', 2: 'q', 3: 's', 4: 'h', 5: 'k', 6: 'b', 7: 'ä', 8: 'ł', 9: 'd', 10: 'ß', 11: 't', 12: 'ó', 13: 'ì', 14: 'g', 15: 'a', 16: 'm', 17: 'è', 18: 'õ', 19: 'ò', 20: 'ń', 21: 'j', 22: 'ú', 23: 'ż', 24: 'ą', 25: "'", 26: ':', 27: 'ś', 28: 'l', 29: 'f', 30: 'v', 31: 'ç', 32: '/', 33: 'i', 34: 'ñ', 35: '1', 36: 'y', 37: 'c', 38: 'p', 39: 'u', 40: 'é', 41: 'ü', 42: 'e', 43: 'á', 44: 'r', 45: 'ê', 46: 'w', 47: 'à', 48: 'z', 49: '-', 50: 'n', 51: 'ö', 52: 'x', 53: 'í'}
17


In [186]:
class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, surname):
    surname_t = torch.zeros(self.vocab.max_seq_len, dtype=torch.int64)
    for i, token in enumerate(surname):
        if i >= self.vocab.max_seq_len:
            break
        surname_t[i] = self.vocab.token_to_idx.get(token, 0)
    return surname_t

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    surname = self.X.iloc[idx]
    label = self.y.iloc[idx]
    surname_t = self.vectorize(surname)
    return surname_t, label

In [187]:
train_dataset = SurnamesDataset(X_train, y_train, vocab)
test_dataset = SurnamesDataset(X_test, y_test, vocab)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [188]:
train_dataset[1][0]

tensor([44, 15, 46, 28, 33, 50, 14,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs):
    model.to(device)
    train_losses, test_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss, test_loss = 0, 0
        for inputs, labels in train_dataloader:
            x = inputs.to(device)
            y = labels.to(device)

            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss += loss.item()

        # Валидация на val_loader
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item()

        train_losses.append(train_loss/len(train_dataloader))
        test_losses.append(test_loss/len(test_dataloader))

        print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}')

def evaluate_model(model, dataloader):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for surnames, labels in dataloader:
            x = surnames.to(device)
            y = labels.to(device)

            logits = model(x)
            _, predicted = torch.max(logits, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.5f}')

def predict(model, dataset, surname):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    model.eval()
    with torch.no_grad():
        vectorized = dataset.vectorize(surname)
        tensor = vectorized.unsqueeze(0).to(device)

        logits = model(tensor)

        probs = torch.softmax(logits, dim=1).squeeze()
        top3_probs, top3_indices = torch.topk(probs, k=3)
        print(top3_probs, top3_indices)

        top3_nationalities = _[top3_indices.detach().cpu().numpy()]
        print(f'{surname}: {top3_nationalities.values[0]} ({top3_probs[0].item():.4f}), {top3_nationalities.values[1]} ({top3_probs[1].item():.4f}), {top3_nationalities.values[2]} ({top3_probs[2].item():.4f})')

In [190]:
class SurnameClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, use_unit_vectors=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim if not use_unit_vectors else vocab_size, 32, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        # print(f"embedded shape {embedded.shape}")

        conv1_out = self.pool1(torch.relu(self.conv1(embedded)))
        # print(f"conv1 + maxpool shape {conv1_out.shape}")

        conv2_out = self.pool2(torch.relu(self.conv2(conv1_out)))
        # print(f"conv2 + maxpool shape {conv2_out.shape}")

        flattened = self.flatten(conv2_out)
        # print(f"flattened shape {flattened.shape}")

        logits = self.fc(flattened)
        # print(f"fc shape: {logits.shape}")
        return logits

In [191]:
vocab.vocab_len

54

In [192]:
model = SurnameClassifier(vocab_size=vocab.vocab_len,
                           embedding_dim=vocab.max_seq_len,
                           output_dim=len(set(y_train)),
                           use_unit_vectors=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [193]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Количество обучаемых параметров: {total_params}")

Количество обучаемых параметров: 11112


In [194]:
model.forward(train_dataset.vectorize("valyaev").unsqueeze(0).to(device))

tensor([[-0.2979, -0.3806, -0.0603,  0.0929, -0.1632, -0.1362, -0.3917, -0.0880,
          0.0042, -0.1344,  0.1362, -0.1797,  0.1474, -0.0490, -0.3407,  0.1323,
          0.1141, -0.0716]], grad_fn=<AddmmBackward0>)

In [195]:
train_model(model, train_dataloader, test_dataloader, criterion, optimizer, 15)

Epoch 1, Train Loss: 2.2255, Test Loss: 2.0523
Epoch 2, Train Loss: 1.9377, Test Loss: 1.8590
Epoch 3, Train Loss: 1.7386, Test Loss: 1.6957
Epoch 4, Train Loss: 1.5956, Test Loss: 1.5818
Epoch 5, Train Loss: 1.5027, Test Loss: 1.5047
Epoch 6, Train Loss: 1.4231, Test Loss: 1.4392
Epoch 7, Train Loss: 1.3632, Test Loss: 1.4011
Epoch 8, Train Loss: 1.3093, Test Loss: 1.3373
Epoch 9, Train Loss: 1.2524, Test Loss: 1.3082
Epoch 10, Train Loss: 1.2094, Test Loss: 1.2758
Epoch 11, Train Loss: 1.1734, Test Loss: 1.2495
Epoch 12, Train Loss: 1.1309, Test Loss: 1.2434
Epoch 13, Train Loss: 1.1051, Test Loss: 1.2135
Epoch 14, Train Loss: 1.0746, Test Loss: 1.1806
Epoch 15, Train Loss: 1.0410, Test Loss: 1.1735


In [196]:
evaluate_model(model, test_dataloader)

Test Accuracy: 0.66530


In [197]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
X_batch, y_batch = next(iter(test_loader))
predictions = model(X_batch.to(device)).argmax(dim=1).cpu().detach()
print(classification_report(y_batch, predictions))

              precision    recall  f1-score   support

           0       0.59      0.88      0.70       567
           1       0.00      0.00      0.00        36
           2       0.82      0.99      0.90       346
           3       0.82      0.74      0.78       482
           4       0.83      0.73      0.78       161
           5       0.44      0.89      0.59        36
           6       0.37      0.62      0.46       108
           7       0.43      0.11      0.18        81
           8       0.56      0.12      0.20        41
           9       0.42      0.16      0.23       118
          10       0.45      0.16      0.23        32
          11       0.17      0.02      0.03        57
          12       0.00      0.00      0.00        25
          13       0.83      0.10      0.18        49
          14       0.00      0.00      0.00        15
          15       0.50      0.07      0.12        15
          16       0.00      0.00      0.00        14
          17       0.00    

C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [198]:
predict(model, train_dataset, 'valyaev')

tensor([0.9618, 0.0258, 0.0068]) tensor([3, 7, 0])
valyaev: Russian (0.9618), Czech (0.0258), English (0.0068)


In [199]:
predict(model, train_dataset, 'balinyan')

tensor([0.5012, 0.3599, 0.0504]) tensor([3, 0, 8])
balinyan: Russian (0.5012), English (0.3599), Irish (0.0504)


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [26]:
with open("polarity/positive_reviews.txt") as f:
    positive_reviews = sent_tokenize(f.read())

with open("polarity/negative_reviews.txt") as f:
    negative_reviews = sent_tokenize(f.read())

In [27]:
len(positive_reviews), len(negative_reviews)

(7267, 7091)

In [28]:
reviews_df = pd.DataFrame()

reviews_df["text"] = positive_reviews + negative_reviews
reviews_df["category"] = [1 for i in range(len(positive_reviews))] + [0 for i in range(len(negative_reviews))]

reviews_df

,text,category
0,"simplistic , silly and tedious .",1
1,"it's so laddish and juvenile , only teenage bo...",1
2,exploitative and largely devoid of the depth o...,1
3,[garbus] discards the potential for pathologic...,1
4,a visually flashy but narratively opaque and e...,1
...,...,...
14353,may prove to be [tsai's] masterpiece .,0
14354,mazel tov to a film about a family's joyous li...,0
14355,standing in the shadows of motown is the best ...,0
14356,it's nice to see piscopo again after all these...,0


In [29]:
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    text = ''.join([' ' if not char.isalpha() and char not in ['.', ',', '!', '?'] else char for char in text])

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

reviews_df["text"] = reviews_df["text"].apply(lambda x: preprocess_text(x))
reviews_df

,text,category
0,"simplistic , silly and tedious .",1
1,"it s so laddish and juvenile , only teenage bo...",1
2,exploitative and largely devoid of the depth o...,1
3,garbus discard the potential for pathological ...,1
4,a visually flashy but narratively opaque and e...,1
...,...,...
14353,may prove to be tsai s masterpiece .,0
14354,mazel tov to a film about a family s joyous li...,0
14355,standing in the shadow of motown is the best k...,0
14356,it s nice to see piscopo again after all these...,0


In [30]:
reviews_df.iloc[9152]["text"]

'it s the perfect kind of film to see when you don t want to use your brain .'

In [31]:
from sklearn.model_selection import train_test_split

X = reviews_df['text'].str.lower()
y = reviews_df['category']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [32]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = {}
    self.token_to_idx = {}
    self.vocab_len = 0
    self.max_seq_len = 0

    for item in data:
      self.max_seq_len = max(self.max_seq_len, len(item))

    # Добавляем токен для неизвестных слов
    self.idx_to_token = {0: '<UNK>'}
    self.token_to_idx = {'<UNK>': 0}
    self.vocab_len += 1
    # Получаем список всех слов в данных
    all_words = [word for sentence in data for word in word_tokenize(sentence)]

    # Строим словарь
    for word in all_words:
        if word not in self.token_to_idx:
            self.idx_to_token[self.vocab_len] = word
            self.token_to_idx[word] = self.vocab_len
            self.vocab_len += 1

    # Преобразуем данные в последовательности индексов токенов
    self.data = []
    for sentence in data:
        tokens = [self.token_to_idx.get(word, self.token_to_idx['<UNK>']) for word in word_tokenize(sentence)]
        self.data.append(tokens)

vocab = Vocab(X)
vocab.vocab_len

16510

In [33]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''
    vec = torch.zeros(self.vocab.max_seq_len, dtype=torch.int64)


    for i, word in enumerate(word_tokenize(review)):

      if i >= self.vocab.max_seq_len:
        break

      vec[i] = self.vocab.token_to_idx.get(word, 0)

    return vec

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    vec = self.vectorize(self.X[idx])
    label = self.y[idx]
    return vec, label

In [34]:
train_dataset = ReviewDataset(X_train, y_train, vocab)
test_dataset = ReviewDataset(X_test, y_test, vocab)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [50]:
class ReviewClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 64, kernel_size=3)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(12928, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        # print(f"embedded shape: {embedded.shape}")
        # print(self.conv1(embedded).shape)
        conv1_out = self.pool(torch.relu(self.conv1(embedded)))
        # print(conv1_out.shape)
        conv2_out = self.pool(torch.relu(self.conv2(conv1_out)))
        # conv3_out = self.pool(torch.relu(self.conv3(conv2_out)))
        # print(conv2_out.shape)
        flattened = self.flatten(conv2_out)
        # print(flattened.shape)
        logits = self.fc(flattened)
        return logits

In [51]:
vocab.max_seq_len

413

In [52]:
model = ReviewClassifier(vocab_size=vocab.vocab_len,
                           embedding_dim=vocab.max_seq_len,
                           output_dim=len(set(y_train)))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [53]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Количество обучаемых параметров: {total_params}")

Количество обучаемых параметров: 6948552


In [54]:
train_model(model, train_dataloader, test_dataloader, criterion, optimizer, 10)

Epoch 1, Train Loss: 0.7138, Test Loss: 0.6933
Epoch 2, Train Loss: 0.6935, Test Loss: 0.6927
Epoch 3, Train Loss: 0.6929, Test Loss: 0.6942
Epoch 4, Train Loss: 0.6920, Test Loss: 0.6946
Epoch 5, Train Loss: 0.6837, Test Loss: 0.6775
Epoch 6, Train Loss: 0.6272, Test Loss: 0.6564
Epoch 7, Train Loss: 0.5291, Test Loss: 0.6726
Epoch 8, Train Loss: 0.4253, Test Loss: 0.7524
Epoch 9, Train Loss: 0.3460, Test Loss: 0.7354
Epoch 10, Train Loss: 0.2837, Test Loss: 0.7961


In [55]:
evaluate_model(model, test_dataloader)

Test Accuracy: 0.67061


In [56]:
rating_labels = ["Positive", "Negative"]

def predict(model, dataset, review):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    model.eval()
    with torch.no_grad():
        vectorized = dataset.vectorize(review)
        tensor = vectorized.unsqueeze(0).to(device)
        logits = model(tensor)
        probs = torch.softmax(logits, dim=1).squeeze()
        print(probs)
        print(f'{rating_labels[probs.argmax()]} ({probs.max():.4f}), {rating_labels[probs.argmin()]} ({probs.min():.4f}) \n{review}')

In [57]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

model.to("cpu")

test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
X_batch, y_batch = next(iter(test_loader))
predictions = model(X_batch).argmax(dim=1).cpu().detach()
print(classification_report(y_batch, predictions))

              precision    recall  f1-score   support

           0       0.63      0.78      0.70      1413
           1       0.72      0.57      0.64      1459

    accuracy                           0.67      2872
   macro avg       0.68      0.67      0.67      2872
weighted avg       0.68      0.67      0.67      2872



In [58]:
predict(model, train_dataset, "I had a terrible experience at this restaurant. The staff was rude and the food was overpriced for the quality.")

tensor([0.1447, 0.8553])
Negative (0.8553), Positive (0.1447) 
I had a terrible experience at this restaurant. The staff was rude and the food was overpriced for the quality.


In [59]:
predict(model, train_dataset, "This restaurant is simply amazing! The food is delicious and the service is outstanding.")

tensor([0.9951, 0.0049])
Positive (0.9951), Negative (0.0049) 
This restaurant is simply amazing! The food is delicious and the service is outstanding.


In [60]:
predict(model, train_dataset, "The menu at this restaurant is very limited and the food is nothing special. I wouldn't go back.")

tensor([0.5827, 0.4173])
Positive (0.5827), Negative (0.4173) 
The menu at this restaurant is very limited and the food is nothing special. I wouldn't go back.


In [61]:
predict(model, train_dataset, "I can't say enough good things about this restaurant. It's the perfect place for a romantic dinner or a night out with friends.")

tensor([0.9062, 0.0938])
Positive (0.9062), Negative (0.0938) 
I can't say enough good things about this restaurant. It's the perfect place for a romantic dinner or a night out with friends.


In [62]:
predict(model, train_dataset, "I had the best dining experience in this restaurant. The ambiance is perfect and the staff is very friendly.")

tensor([0.6659, 0.3341])
Positive (0.6659), Negative (0.3341) 
I had the best dining experience in this restaurant. The ambiance is perfect and the staff is very friendly.
